In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import os

# proxyの設定．
# keras.datasetsでは，datasetを直接ダウンロードするので，学内マシンからは通常必要．
os.environ["http_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["https_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"      # "0":GPU0, "1":GPU1, "0,1":GPUを2つとも使用

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from skimage import feature,filters
from PIL import Image
import sys

import cv2

# ResNet50 による 1000種類分類
import torchvision.models as models
import pickle
import glob


In [2]:
! ls -l ~/.cache/torch/checkpoints/ 

total 1206624
-rw------- 1 yamamoto-k YANAI_LAB 102502400 Apr 27 18:13 resnet50-19c8e357.pth
-rw------- 1 yamamoto-k YANAI_LAB   4966400 Apr 29 20:54 squeezenet1_1-f364aa15.pth
-rw------- 1 yamamoto-k YANAI_LAB 553433881 Apr 25 19:43 vgg16-397923af.pth
-rw------- 1 yamamoto-k YANAI_LAB 574673361 Apr 29 13:53 vgg19-dcbb9e9d.pth


In [4]:
net = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana')

Downloading: "https://github.com/milesial/Pytorch-UNet/archive/master.zip" to /home/yanai-lab/yamamoto-k/.cache/torch/hub/master.zip


In [8]:
!python3 Pytorch-UNet/predict.py -i carvana_1.jpeg -o output.jpg -m unet_carvana_scale1_epoch5.pth

![carvana_1](carvana_1.jpeg) 

![carvana_1](output.jpg) 

pretrained model を探すためにネットの海を探し回っていた。結果的にUNetのpretrained modelを用いいたが、epoch数は少ない。それっぽいマスクはできている。
U-netはエンコーダーでデータを圧縮していって、圧縮したデータをデコーダーで展開させる時にエンコーダの特徴量を埋め込むことで、行っている。

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits
        
 構造が単純でなのはFCNで全結合層をConvolution層に置き換えるだけであったので、構造を真似すれば構成できる。
 
それぞれで、領域分割が何を目的として作られているかで、性質が変わりそうなので、そういった背景も考慮しながらモデルを選ぶのもいいかもしれない。


https://github.com/meetshah1995/pytorch-semseg<br>
https://github.com/zijundeng/pytorch-semantic-segmentation<br>
https://github.com/delta-onera/segnet_pytorch<br>

https://pytorch.org/hub/pytorch_vision_fcn_resnet101/

https://github.com/milesial/Pytorch-UNet